<a href="https://colab.research.google.com/github/Timliuw/DP-Tim/blob/main/CI_Tim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import pandas as pd
import uuid

In [136]:
#for experiments purpose
import gspread
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import files
# uploaded = files.upload()
import os
os.listdir()

scope = [
    "https://spreadsheets.google.com/feeds",  # Google Sheets API
    "https://www.googleapis.com/auth/drive"   # Google Drive API
]

# Trigger the file upload dialog
try:
    creds = ServiceAccountCredentials.from_json_keyfile_name("sublime-mission-447319-c2-0df1612796a8.json", scope)
except:
   uploaded = files.upload()

creds = ServiceAccountCredentials.from_json_keyfile_name("sublime-mission-447319-c2-0df1612796a8.json", scope)
client = gspread.authorize(creds)

spreadsheet = client.open_by_key("1yVVt5ACdBbjsZ_7IiRkvPGIHmpKe8mFXcvyL0PVP2cE")

worksheet1 = spreadsheet.get_worksheet(0)
worksheet = worksheet1
worksheet2 = spreadsheet.get_worksheet(1)

def get_color(curr_value, prev_value):
    if curr_value < 0.6 * prev_value:
        return {"red": 0.6, "green": 1.0, "blue": 0.6}  # very green
    elif curr_value < prev_value:
        return {"red": 0.8, "green": 1.0, "blue": 0.8}  # green
    elif curr_value > 1.4 * prev_value:
        return {"red": 1.0, "green": 0.6, "blue": 0.6}  # very red
    elif curr_value > prev_value:
        return {"red": 1.0, "green": 0.8, "blue": 0.8}  # red
    return None

def write_to_sheet(worksheet, row_data, output_mode=0, compare=False):
    # Append the new row to the sheet
    try:
        worksheet.append_row(row_data)
    except Exception as e:
        print(f"Error occurred: {e}")
        print(f"Row data: {row_data}")
    # Format colour according to the comparing results
    if compare:
        pre_row_index = -2
        if (output_mode == 2): pre_row_index = -16
        sheet_data = worksheet.get_all_values()
        last_row_index = len(sheet_data)
        if last_row_index > 1:
            last_row = sheet_data[-1]
            prev_row = sheet_data[pre_row_index]
            for col_index, (curr_value, prev_value) in enumerate(zip(last_row, prev_row), start=1):
                try:
                    curr_value = float(curr_value)
                    prev_value = float(prev_value)
                    if (worksheet.row_values(1)[col_index-1] == "est median"):
                      prev_value = abs(true_med - prev_value)
                      curr_value = abs(true_med - curr_value)
                    color = get_color(curr_value, prev_value)
                    if color:
                        worksheet.format(f"{chr(64 + col_index)}{last_row_index}", {"backgroundColor": color})
                except:
                    continue

In [6]:
def discretize(D, b):
    n = len(D)
    discreteD = np.zeros(n)
    for i in range(n):
        discreteD[i] = int(D[i]/b)
    return discreteD

def discrete(D):
    n = len(D)
    newD = np.zeros(n)
    previous = -1
    for i in range(n):
        if D[i]!=previous:
            newD[i] = n*D[i]
            #print(newD[i])
            previous = D[i]
        elif D[i]==previous:
            newD[i] = newD[i-1]+1

    return newD

def count(D, a):
    counter = 0
    n = len(D)
    low = 0
    up=n-1
    mid = int((low+up)/2)
    while True:
        if D[mid]>a:
            up=mid
            mid = int((low+up)/2)
        if D[mid]<a:
            low=mid
            mid = int((low+up)/2)
        if D[mid]==a:
            i=0
            while D[mid+i]==a:
                i+=1
            return mid+i

def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

def F(x):
    return 1/2+1/(4*math.pi)*(math.log(abs(2*x**2+2*math.sqrt(2)*x+2)/abs(abs(2*x**2-2*math.sqrt(2)*x+2)))+2*math.atan(math.sqrt(2)*x+1)+2*math.atan(math.sqrt(2)*x-1))
def inver_F(y):
    #find the solution of F(x)=y
    #Find between -1000000 and 1000000 because F(1000000)=1.0 in python
    if y>1/2:
        low =0.0
        high = 1000000.0
        mid = (high+low)/2
        while abs(high-low)>0.0000001:
            if F(mid)>y:
                high=mid
            elif F(mid)==y:
                return mid
            else:
                low = mid
            mid = (high+low)/2

        return high
    if y==1/2:
        return 0

def CauchyNoise():
    a = random.uniform(0.5,1)
    b = inver_F(a)
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

In [7]:
def constructu(eps, a,b, D):
    global u
    global l
    global weight
    #[a,b] denotes range
    n = len(D)
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+2)#score changing point
    l[0] = a
    for i in range(n+1):
        if i<=int(n/2):
            u[i] = -int(n/2)-1+i
            l[i+1]= D[i]
        #u[int(n/2+1)] = 0
        #l[int(n/2+2)] = D[int(n/2)]
        if i>int(n/2):
            u[i] = int(n/2)+1-i
            l[i]= D[i-1]

    l[n+1] = b
    u[n+1] = -n-1-u[0]
    weight = []
    for i in range(int(n/2)+1):
        weight.append((l[i+1]-l[i])*math.pow(np.e, eps*u[i]/2))
    weight.append(1)
    for i in range(int(n/2)+1,n+1):
        weight.append((l[i]-l[i-1])*math.pow(np.e, eps*u[i]/2))

    totalWeight = sum(weight)
    weight = weight/totalWeight
    #print(len(weight))

def EMMedian_new():
    i = np.random.choice(list(range(len(l))), p=weight)
    if i==int(n/2)+1:
        return int(l[i]/n)
    if i<int(n/2)+1:
        return int(np.random.randint(l[i], l[i+1],dtype=np.int64)/n)
    if i>int(n/2)+1:
        return int(np.random.randint(l[i-1], l[i],dtype=np.int64)/n)

def constructu_CI(eps, beta, N):
    global u1
    global weight1
    global u2
    global weight2
    global factor

    factor = int(8/eps*np.log(4*n*N/beta))
    print("factor is "+str(factor))
    u1 = np.zeros(n+2)
    u2 = np.zeros(n+2)
    for i in range(n+2):
        if i<=int(n/2)+1:
            u1[i] = -abs(u[i]+factor)
            u2[i] = u[i]-factor
        else:
            u1[i] = u[i]-factor
            u2[i] = -abs(u[i]+factor)

    idx = int(n/2)+1-factor
    weight1 = []
    for i in range(idx):
        weight1.append((l[i+1]-l[i])*math.pow(np.e, eps*u1[i]/4))
    weight1.append(1)
    for i in range(idx,n+1):
        weight1.append((l[i]-l[i-1])*math.pow(np.e, eps*u1[i]/4))

    totalWeight1 = sum(weight1)
    weight1 = weight1/totalWeight1

    idx = int(n/2)+1+factor
    weight2 = []
    for i in range(idx):
        weight2.append((l[i+1]-l[i])*math.pow(np.e, eps*u2[i]/4))
    weight2.append(1)
    for i in range(idx,n+1):
        weight2.append((l[i]-l[i-1])*math.pow(np.e, eps*u2[i]/4))

    totalWeight2 = sum(weight2)
    weight2 = weight2/totalWeight2

def EMMedianCI():
    i1 = np.random.choice(list(range(len(l))), p=weight1)
    if i1==int(n/2)+1-factor:
        x1= int(l[i1]/n)
    if i1<int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1], l[i1+1],dtype=np.int64)/n)
    if i1>int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1-1], l[i1],dtype=np.int64)/n)

    i2 = np.random.choice(list(range(len(l))), p=weight2)
    if i2==int(n/2)+1+factor:
        x2= int(l[i2]/n)
    if i2<int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2], l[i2+1],dtype=np.int64)/n)
    if i2>int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2-1], l[i2],dtype=np.int64)/n)

    est = EMMedian_new()
    indicator=0
    if D[int(n/2)]<=x2 and D[int(n/2)]>=x1:
        indicator=1

    return x1,x2 ,indicator,est

In [8]:
def SVTquantile(eps, beta, D, tau, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    T = tau

    for i in range(rad+1):
        Ti = T-4/eps*np.log(2*(i+1)**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return i
def SVTmedian(eps, beta, D, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    n=len(D)
    T = int(n/2)

    for i in range(rad+1):
        Ti = T+TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return i

def SVTMedian(eps, beta, D):
    n = len(D)
    rad = 1000000
    Q = SVTmedian(eps, beta, D, rad)

    factor = int(16/eps*math.log(2*(2*rad+1)**2 *math.pi**2/(3*beta))+8/eps*math.log(4/beta))+1
    ##Then run SVT, do not double
    ##find the n/2 +- factor quantile
    T1 = int(n/2)-1
    T2 = int(n/2)+factor
    XLeft = SVTquantile(eps/2, beta/2, D, T1, rad)
    #print(XLeft)
    XRight = SVTquantile(eps/2, beta/2, D, T2, rad)
    #print(XRight)
    indicator=0
    if D[int(n/2)]<=Q+ XRight and D[int(n/2)]>=XLeft:
        indicator=1

    return Q, XLeft,XRight,indicator

In [9]:
def smoothMedian(eps, beta, D,delta=0):
    #given D, first compute S(D)
    n = len(D)
    m = int(n/2)
    rad = 1000000
    SS = 0
    if delta==0:
        alpha = eps/20
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))

        Q = D[int(n/2)]+20*SS/eps*CauchyNoise()
        #print(Q)
        indicator=0
        factor = 20*noisySS/eps*inver_F(1-beta/4)
        if D[int(n/2)]<=Q+ factor and D[int(n/2)]>=Q-factor:
            indicator=1

        return Q, factor,indicator
    else:
        alpha = eps/(4*math.log(1/delta))
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        #print(noisySS)
        Q = D[int(n/2)]+4*SS/eps*LapNoise()


        indicator=0
        if D[int(n/2)]<=Q+ 4*noisySS/eps*math.log(2/beta) and D[int(n/2)]>=Q-4*noisySS/eps*math.log(2/beta):
            indicator=1

        return Q, 4*noisySS/eps*math.log(2/beta),indicator


In [10]:
# Our Additions (September 27th)
# Exponetial Mechanism + Confidence Intervals

#ci = (2 * sens / eps) * (np.log(domain_size / beta))
#print(ci)

#np.unique(D,return_counts=True)

#vals, counts = np.unique(D,return_counts=True)
#print(vals)
#vals = [0.0] + vals.tolist()
#print(vals)

def basic_EM(eps,beta,D,domain_size):
    print("Results for EM, beta = "+str(beta) + ", epsilon = " + str(eps))
    #print(D)
    #D = D.sort() # D is sorted
    interval_prob = []
    vals, counts = np.unique(D,return_counts=True)
    #print(vals)
    #print(counts)
    rank = 0
    vals = [0.0] + vals.tolist()
    cdf = 0.0
    for i in range(1,len(vals)):
        rank += counts[i-1]
        utility = -1.0* np.abs(rank - (len(D)/2))
        #print(rank)
        #print(utility)
        #print(math.pow(np.e, eps*utility))
        p = (vals[i] - vals[i-1])* math.pow(np.e, eps*utility)
        cdf += p
        interval_prob.append(cdf)
    rand_val = cdf * np.random.uniform(0,1)
    #print(cdf,rand_val)
    for i in range(1,len(vals)):
        if rand_val <= interval_prob[i]:
            sens = 1
            ci = (2 * sens / eps) * (np.log(domain_size / beta))
            med_val = np.random.uniform(vals[i-1],vals[i])

            print("median rank: %d out of %d" %(i,len(vals)) )
            print("range for median rank: (%f,%f)" %(vals[i-1],vals[i]))
            #print("sampled median value: (%f)" % (med_val)) #need to convert from rank to true value
            print("size of confidence interval: %f" %(ci)) # for rank
            break


In [11]:
# November 22nd / November 29th

#1) Learn noisy median o of distribution using Exponential Mechanism
def EM_median(eps,beta,D,domain_size):
    #print("Results for EM, beta = "+str(beta) + ", epsilon = " + str(eps))
    interval_prob = []
    vals, counts = np.unique(D,return_counts=True)
    rank = 0
    vals = [0.0] + vals.tolist()
    cdf = 0.0
    rank_dict = {}
    for i in range(1,len(vals)):
        rank += counts[i-1]
        rank_dict[vals[i]] = rank
        utility = -1.0* np.abs(rank - (len(D)/2))
        p = (vals[i] - vals[i-1])* math.pow(np.e, eps*utility)
        cdf += p
        interval_prob.append(cdf)
    rand_val = cdf * np.random.uniform(0,1)
    rank_o = 0
    for i in range(1,len(vals)):
        if rand_val <= interval_prob[i]:
            sens = 1
            ci = (2 * sens / eps) * (np.log(domain_size / beta))
            o = np.random.uniform(vals[i-1],vals[i])
            rank_o = rank_dict[vals[i-1]]
            #print("median rank: %d out of %d" %(rank_dict[vals[i-1]],len(vals)) )
            #print("range for median rank: (%f,%f)" %(vals[i-1],vals[i]))
            #print("sampled median value: (%f)" % (o)) #need to convert from rank to true value
            #print("size of confidence interval: %f" %(ci)) # for rank
            break
    return o,rank_o,rank_dict

#2)
def find_rank(rank_dict,val):
    ranks = list(rank_dict.values())
    keys = list(rank_dict.keys())
    output = 0
    for i in range(len(ranks)-1):
        if val < keys[i+1]:
            output = ranks[i]
            break
    #print(rank_dict[output])
    return output#int(output)


def SVT_median(o,rank_o,rank_dict,eps,beta,D,domain_size,T,b_list):
    #print("T: ", T)
    noisy_T = T + np.random.laplace(2/eps)
    #print("noisy_T: ", noisy_T)
    #print("rank_dict", list(rank_dict.values()))
    #print("rank_keys", list(rank_dict.keys()))

    sensitivity = 1
    #b_max = 10
    #step = 1
    bounds = domain_size
    for b in b_list: #range(0,b_max,step):
        perturbed_value = min(abs(find_rank(rank_dict, o+b)-rank_o),abs(find_rank(rank_dict, o-b)-rank_o)) + np.random.laplace(2*sensitivity/eps)
        if perturbed_value > noisy_T:
            bounds = b
            break
    #print(max(o-bounds,0))
    #print(min(o+bounds,domain_size-1))
    return max(o-bounds,0),min(o+bounds,domain_size-1)

##### main function ######

def our_CI():
    C = (2/eps_1) *(np.log(domain_size/beta_1))
    alpha = (8*np.log(m)+np.log(2/beta_2)) / eps_2
    #print("C", C)
    #print("alpha", alpha)
    T = C + alpha
    o,rank_o,rank_dict = EM_median(eps_1,beta_1,D,domain_size)
    lower,upper = SVT_median(o,rank_o,rank_dict,eps_2,beta_1,D,domain_size,T,b_list)
    #print("median:",o)
    #print("The bounds are:")
    #print('[{},{}]'.format(lower,upper))
    indi = -1 #if CI contains true med
    if true_med < upper and true_med > lower:
        indi = 1
    else:
        indi = 0
    return lower,upper,indi,o


In [149]:
#Test for our_CI
def test_ours(compare = False):
  len_eps = []
  err_eps = []
  est_median = []
  times = []
  ranks = []
  rank_errors_1 = [] # based on n/2
  rank_errors_2 = [] # based on find_rank(rank_dict,true_med)
  # est_list = []
  med_rank = int(find_rank(rank_dict, true_med))
  for i in range(len(beta)):
      print("Results for EM based median, beta = "+str(beta[i]))
      errors = []
      lengths = []
      correct_count = 0

      for j in range(num_repeat):
          start = time.time()
          xl,xr,indi,est = our_CI()#EMMedianCI()
          end = time.time()
          times.append(end-start)
          correct_count+=indi
          lengths.append((xr-xl)/(2))
          est = (xr+xl)/2
          est_median.append(est)
          errors.append(abs(est-true_med))
          est_rank = int(find_rank(rank_dict, est))
          ranks.append(est_rank)
          rank_errors_1.append(int(abs(n/2 - est_rank)))
          rank_errors_2.append(int(abs(med_rank - est_rank)))
          # est_list.append(est)
      original_errors = errors
      errors = np.array(errors, dtype=object)
      correct_rate = correct_count/num_repeat
      avgLength = sum(lengths)/num_repeat
      errors.sort()
      errorQuantile = errors[int(num_repeat*(1-beta[i]))]
      avg_error = np.average(errors)
      avgTime = sum(times)/len(times)


      # output medians and error

      #download errors
      # df = pd.DataFrame(errors)
      # file_name = f"data_error.csv"
      # df.to_csv(file_name, index=False, header=False)
      # files.download(file_name)

      # median_error
      if np.all(errors == errors[0]):
          stddev_based_avg_err = errors[0]
          IQR_based_avg_err = errors[0]
      else:
          std_dev = np.std(errors)
          stddev_based_avg_err = np.average(errors[np.abs(errors - avg_error) < 3 * std_dev])
          q1 = np.percentile(errors, 25)
          q3 = np.percentile(errors, 75)
          iqr = q3 - q1
          lower_bound = q1 - 1.5 * iqr
          upper_bound = q3 + 1.5 * iqr
          IQR_based_avg_err = np.average(errors[(errors >= lower_bound) & (errors <= upper_bound)])
      # rank_error
      avg_rank_error_1 = np.average(rank_errors_1)
      avg_rank_error_2 = np.average(rank_errors_2)
      print("estimated median value: "+ str(np.average(est_median)))
      print("correct rate = ", correct_rate)
      print("Average CI length = "+str(avgLength))
      print("error quantile is "+ str(errorQuantile))
      print("CI length/ error quantile = "+str(avgLength/errorQuantile))
      print("Average Time to find CI = " + str(avgTime))
      if (output_mode == 0 or output_mode == 2):
        row_data = [
            "our_CI",
            data_uuid,
            n,
            domain_size,
            sens,
            beta[i],
            eps[i],
            num_repeat,
            true_med,
            np.average(est_median),
            correct_rate,
            avgLength,
            errorQuantile,
            avg_error,
            stddev_based_avg_err,
            IQR_based_avg_err,
            avg_rank_error_1,
            avg_rank_error_2,
            avgLength / errorQuantile,
            avgTime,
            b_list_step,
            beta_1,
            beta_2,
            eps_1,
            eps_2
        ]
        write_to_sheet(worksheet1, row_data, output_mode=output_mode, compare=compare)

      # New! Check all medians and errors
      if (output_mode == 1 or output_mode == 2):
        sorted_med = sorted(est_median)
        est_median.insert(0, "Our est_median")
        sorted_med.insert(0, "Sorted Our est_median")

        sorted_errors = sorted(original_errors)
        original_errors.insert(0, "Our errors")
        sorted_errors.insert(0, "Sorted Our errors")

        sorted_ranks = sorted(ranks)
        ranks.insert(0, "Our ranks")
        sorted_ranks.insert(0, "Sorted Our ranks")

        sorted_re_1 = sorted(rank_errors_1)
        sorted_re_2 = sorted(rank_errors_2)
        rank_errors_1.insert(0, "Our rank_errors(n/2)")
        rank_errors_2.insert(0, "Our rank_errors(find)")
        sorted_re_1.insert(0, "Sorted Our rank_errors(n/2)")
        sorted_re_2.insert(0, "Sorted Our rank_errors(find)")

        data_uid_row = ["data:", data_uuid, "method:", "Our_CI"]
        true_med_row = ["true median"]+[true_med]*num_repeat
        n_over_2_row = ["true n/2"] + [n/2]*num_repeat
        med_rank_row = ["true med_rank(find)"] + [med_rank]*num_repeat

        worksheet.append_row(data_uid_row)
        worksheet.append_row(est_median)
        worksheet.append_row(true_med_row)
        worksheet.append_row(original_errors)
        # write_to_sheet(worksheet1, errors, output_mode=output_mode, compare=compare)
        worksheet.append_row(ranks)
        worksheet.append_row(n_over_2_row)
        worksheet.append_row(rank_errors_1)
        # write_to_sheet(worksheet1, rank_errors_1, output_mode=output_mode, compare=compare)
        worksheet.append_row(med_rank_row)
        worksheet.append_row(rank_errors_2)
        # write_to_sheet(worksheet1, rank_errors_2, output_mode=output_mode, compare=compare)
        worksheet.append_row(sorted_med)
        worksheet.append_row(sorted_ranks)
        worksheet.append_row(sorted_errors)
        worksheet.append_row(sorted_re_1)
        worksheet.append_row(sorted_re_2)


      len_eps.append(avgLength)
      err_eps.append(errorQuantile)
  print(len_eps)
  print(err_eps)

In [13]:
# Used to test difference between est and est_median

# if est_median != est_list:
#     print("The two lists are different. The differences are as follows:")

#     # Check each position for differing elements
#     for j, (a, b) in enumerate(zip(est_median, est_list)):
#         if a != b:
#             print(f"Index {j}: est_median = {a}, est_list = {b}")

# else:
#     print("The two lists are the same")

# avg_error = np.average(errors)

In [170]:
def test_theirs(compare = False):
  len_eps = []
  err_eps = []
  est_median = []
  times = []
  ranks = []
  med_rank = int(find_rank(rank_dict, true_med))
  rank_errors_1 = [] # based on n/2
  rank_errors_2 = [] # based on find_rank(rank_dict,true_med)
  for i in range(len(beta)):
      print("Results for EM based median, beta = "+str(beta[i]))
      errors = []
      lengths = []
      correct_count = 0
      n = len(newD)
      rad = 10000000
      left=-rad
      constructu(eps[i], left*n,rad*n, newD)

      constructu_CI(eps[i],beta[i], rad)
      for j in range(num_repeat):
          start = time.time()
          xl,xr,indi,est = EMMedianCI()
          end = time.time()
          times.append(end-start)
          correct_count+=indi
          lengths.append((xr-xl)/(2))
          est = (xr+xl)/2
          est_median.append(est)
          errors.append(abs((xr+xl)/2-true_med))
          est_rank = int(find_rank(rank_dict, est))
          ranks.append(est_rank)
          rank_errors_1.append(int(abs(n/2 - est_rank)))
          rank_errors_2.append(int(abs(med_rank - est_rank)))

      original_errors = errors
      correct_rate = correct_count/num_repeat
      avgLength = sum(lengths)/num_repeat
      errors.sort()
      errorQuantile = errors[int(num_repeat*(1-beta[i]))]
      avgTime = sum(times)/len(times)
      # new

      errors = np.array(errors)
      avg_error = np.average(errors)
      if np.all(errors == errors[0]):
          stddev_based_avg_err = errors[0]
          IQR_based_avg_err = errors[0]
      else:
          std_dev = np.std(errors)
          stddev_based_avg_err = np.average(errors[np.abs(errors - avg_error) < 3 * std_dev])

          q1 = np.percentile(errors, 25)
          q3 = np.percentile(errors, 75)
          iqr = q3 - q1
          lower_bound = q1 - 1.5 * iqr
          upper_bound = q3 + 1.5 * iqr
          IQR_based_avg_err = np.average(errors[(errors >= lower_bound) & (errors <= upper_bound)])

      #rank_avg
      avg_rank_error_1 = np.average(rank_errors_1)
      avg_rank_error_2 = np.average(rank_errors_2)
      print("Filtered Errors for IQR_based_avg_err:", IQR_based_avg_err)

      print("estimated median value: "+ str(np.average(est_median)))
      print("correct rate = " +str(correct_count/num_repeat))
      print("Average CI length = "+str(avgLength))
      print("error quantile is "+ str(errorQuantile))
      print("CI length/ error quantile = "+str(avgLength/errorQuantile))
      print("Average Time to find CI = " + str(sum(times)/len(times)))
      # print("Errors:", errors)
      # print("Standard Deviation:", std_dev)
      # print("Filtered Errors for stddev_based_avg_err:", errors[np.abs(errors - avg_error) < 3 * std_dev])

      # print("Q1:", q1, "Q3:", q3)
      # print("IQR:", iqr)
      # print("Lower Bound:", lower_bound, "Upper Bound:", upper_bound)
      if (output_mode == 0 or output_mode == 2):
        row_data = [
            "EM_CI",
            data_uuid,
            n,
            domain_size,
            sens,
            beta[i],
            eps[i],
            num_repeat,
            true_med,
            np.average(est_median),
            correct_rate,
            avgLength,
            errorQuantile,
            avg_error,
            stddev_based_avg_err,
            IQR_based_avg_err,
            avg_rank_error_1,
            avg_rank_error_2,
            avgLength / errorQuantile,
            avgTime,
            b_list_step,
            beta_1,
            beta_2,
            eps_1,
            eps_2
        ]
        write_to_sheet(worksheet1, row_data, compare = compare)

      # New! Check all medians and errors
      if (output_mode == 1 or output_mode == 2):
        sorted_med = sorted(est_median)
        est_median.insert(0, "EM est_median")
        sorted_med.insert(0, "Sorted EM est_median")

        sorted_errors = sorted(original_errors)
        original_errors.insert(0, "EM errors")
        sorted_errors.insert(0, "Sorted EM errors")

        sorted_ranks = sorted(ranks)
        ranks.insert(0, "EM ranks")
        sorted_ranks.insert(0, "Sorted EM ranks")

        sorted_re_1 = sorted(rank_errors_1)
        sorted_re_2 = sorted(rank_errors_2)
        rank_errors_1.insert(0, "EM rank_errors(n/2)")
        rank_errors_2.insert(0, "EM rank_errors(find)")
        sorted_re_1.insert(0, "Sorted EM rank_errors(n/2)")
        sorted_re_2.insert(0, "Sorted EM rank_errors(find)")

        data_uid_row = ["data:", data_uuid, "method:", "EM_CI"]
        true_med_row = ["true median"]+[true_med]*num_repeat
        n_over_2_row = ["true n/2"] + [n/2]*num_repeat
        med_rank_row = ["true med_rank(find)"] + [med_rank]*num_repeat

        worksheet.append_row(data_uid_row)
        worksheet.append_row(est_median)
        worksheet.append_row(true_med_row)

        worksheet.append_row(original_errors)
        # write_to_sheet(worksheet1, original_errors, output_mode=output_mode, compare=compare)
        worksheet.append_row(ranks)
        worksheet.append_row(n_over_2_row)
        worksheet.append_row(rank_errors_1)
        # write_to_sheet(worksheet1, rank_errors_1, output_mode=output_mode, compare=compare)
        worksheet.append_row(med_rank_row)
        worksheet.append_row(rank_errors_2)
        # write_to_sheet(worksheet1, rank_errors_2, output_mode=output_mode, compare=compare)
        worksheet.append_row(sorted_med)
        worksheet.append_row(sorted_ranks)
        worksheet.append_row(sorted_errors)
        worksheet.append_row(sorted_re_1)
        worksheet.append_row(sorted_re_2)



      len_eps.append(avgLength)
      err_eps.append(errorQuantile)
  print(len_eps)
  print(err_eps)

In [158]:
def readInput(mode):
    import numpy as np

    global D
    global n
    read_data_uuid = "64b7c8bf89f14113bd12f5f15e276546"
    file_paths = {
        0: ("./bank_marketing.csv", 5),
        1: ("./adult.csv", 2),
        2: ("./sample_data/california_housing_test.csv", 3),
        3: ("./sample_data/california_housing_test.csv", 8),
        10: ("./data_"+read_data_uuid+".csv",0)
    }

    if mode not in file_paths:
        raise ValueError("Invalid mode. Choose between 0, 1, or 2.")

    input_path, column_index = file_paths[mode]

    with open(input_path, 'r') as input_file:
        lines = input_file.readlines()
    n = len(lines)
    D = np.zeros(n)
    for i, line in enumerate(lines):
        if i > 0:
            elements = line.split(",")
            try:
                value = float(elements[column_index])
                D[i - 1] = int(value)
            except ValueError:
                print(f"Skipping invalid data at line {i}: {elements[column_index]}")



In [166]:
# Experiment Data Input

dataset_id_dict = {0:"bank_marketing", 1:"adult", 2:"cali_housing_test", 3:"cali_housing_value",
                   10: "old_synthetic_data"}
mode = 0

if (mode == -1):
  n = 4000  # 45#000
  domain_size = 4000  # 40#000
  D = np.random.uniform(0, domain_size, n)
  lowest = 0
  highest = domain_size
else:
  readInput(mode)

  lowest = int(min(D))
  highest = int(max(D))
  print("lowest:",lowest)
  domain_size = highest - lowest

print("Data_size:", n)
print("Domain size:", domain_size)
print(D)

D.sort()
D = discretize(D, 1)
newD = discrete(D)

true_med = np.median(D)  # GET EXACT MEDIAN
print("true median: ", true_med)

# rank_dict
vals, counts = np.unique(D,return_counts=True)
rank_dict = {}
rank = 0
vals = [0.0] + vals.tolist()
for i in range(1,len(vals)):
    rank += counts[i-1]
    rank_dict[vals[i]] = rank
print("true median rank:",n/2)



# Data Persistence
if (mode == -1):
  df = pd.DataFrame(D)
  data_uuid = uuid.uuid4().hex
  file_name = f"data_{data_uuid}.csv"
  df.to_csv(file_name, index=False, header=False)
  files.download(file_name)
  print(f"File saved as {file_name}")
else:
  data_uuid = dataset_id_dict[mode]



lowest: -8019
Data_size: 45212
Domain size: 110146
[2.143e+03 2.900e+01 2.000e+00 ... 6.680e+02 2.971e+03 0.000e+00]
true median:  448.0
true median rank: 22606.0


In [167]:

# CI Parameters
eps = [1]
beta = [0.01]
sens = 1 #fixed
num_repeat=100


# for val, rank in rank_dict.items():
#     print(val,"  ",rank)

# Parameters for our_CI
beta_1 = 0.5 *beta[0]
beta_2 = beta[0] - beta_1
eps_1 = 0.5 * eps[0]
eps_2 = eps[0] - eps_1

b_list_step = 1
if mode == 1: b_list_step = 50
b_list = range(0, domain_size, b_list_step)  # [0,10,20,30,40,50,60,70,80,90,100]
m = len(b_list)



In [172]:
# Test both

# output_mode 0:experiment results,
#        1:medians&ranks
#        2:both
output_mode = 2
test_theirs()
print("-------------------First Test Done--------------------")

if output_mode==2:
  print("wait for 90 secs for the Second Test")
  time.sleep(90)


# test_ours(compare = True)
print("-------------------Second Test Done--------------------")

Results for EM based median, beta = 0.01
factor is 262
Filtered Errors for IQR_based_avg_err: 0.53
estimated median value: 448.53
correct rate = 1.0
Average CI length = 13.14
error quantile is 2.0
CI length/ error quantile = 6.57
Average Time to find CI = 0.012030017375946046
[13.14]
[2.0]
-------------------First Test Done--------------------
wait for 90 secs for the Second Test


KeyboardInterrupt: 

In [153]:
# Test different beta
eps = [1]
eps_1 = 0.5 * eps[0]
eps_2 = eps[0] - eps_1
beta_list = [0.01,0.025,0.05,0.1,0.2]
for beta in beta_list:
  beta = [beta]
  beta_1 = 0.5 * beta[0]
  beta_2 = beta[0] - beta_1
  print(f"Running for beta={beta}")
  test_theirs()
  # test_ours()
  print(f"beta={beta} done")
  time.sleep(20)



Running for beta=[0.01]
Results for EM based median, beta = 0.01
factor is 243
Filtered Errors for IQR_based_avg_err: 8.357142857142858
estimated median value: 2006.09
correct rate = 1.0
Average CI length = 233.55
error quantile is 22.0
CI length/ error quantile = 10.615909090909092
Average Time to find CI = 0.0020871376991271973
[233.55]
[22.0]
beta=[0.01] done
Running for beta=[0.025]
Results for EM based median, beta = 0.025
factor is 235
Filtered Errors for IQR_based_avg_err: 9.742424242424242
estimated median value: 2004.57
correct rate = 1.0
Average CI length = 226.76
error quantile is 16.0
CI length/ error quantile = 14.1725
Average Time to find CI = 0.001212770938873291
[226.76]
[16.0]
beta=[0.025] done
Running for beta=[0.05]
Results for EM based median, beta = 0.05
factor is 230
Filtered Errors for IQR_based_avg_err: 10.675
estimated median value: 2003.845
correct rate = 1.0
Average CI length = 223.155
error quantile is 16.5
CI length/ error quantile = 13.524545454545455
Aver